In [ ]:
# https://github.com/awslabs/autogluon
# AutoGluon is a SOTA AutoML framework

!pip install autogluon.tabular[all]

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
train_data = TabularDataset('../input/tabular-playground-series-mar-2021/train.csv')
test_data = TabularDataset('../input/tabular-playground-series-mar-2021/test.csv')

label = 'target'

In [ ]:
# Fit end-to-end with raw data in under 1 hour, with one line of code
predictor = TabularPredictor(
    label=label,
    eval_metric='roc_auc',
    learner_kwargs={'ignored_columns': ['id']},
).fit(
    train_data,
    presets='best_quality',
    hyperparameters={
        'GBM': [
            {},
            {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
            'GBMLarge',
        ],
    },
    time_limit=3600,
)

In [ ]:
pred_proba = predictor.predict_proba(test_data, as_multiclass=False)
pred_proba.head()

In [ ]:
# Submit
submission = TabularDataset('../input/tabular-playground-series-mar-2021/sample_submission.csv')
submission[label] = pred_proba
submission.to_csv('submission.csv', index=False)
submission.head()

In [ ]:
predictor.leaderboard(silent=True)

In [ ]:
# delete model files, otherwise can cause a bug in Kaggle where it can't find submission.csv file.
predictor.delete_models(models_to_keep=[], dry_run=False)